# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 3 2023 2:42PM,254201,10,SOIVA0000571,Ivaoes Animal Health,Released
1,Jan 3 2023 2:41PM,254200,10,SONSB0002066,"Nextsource Biotechnology, LLC",Released
2,Jan 3 2023 2:41PM,254200,10,SONSB0002065,"Nextsource Biotechnology, LLC",Released
3,Jan 3 2023 2:41PM,254200,10,SOTAH0000874,"Nextsource Biotechnology, LLC",Released
4,Jan 3 2023 2:41PM,254200,10,SONSB0002067,"Nextsource Biotechnology, LLC",Released
5,Jan 3 2023 2:41PM,254200,10,SOTAH0000875,"Nextsource Biotechnology, LLC",Released
6,Jan 3 2023 2:41PM,254200,10,SONSB0002069,"Nextsource Biotechnology, LLC",Released
7,Jan 3 2023 2:36PM,254199,19,ADV80009998,"AdvaGen Pharma, Ltd",Released
8,Jan 3 2023 2:35PM,254198,19,ADV80009999,"AdvaGen Pharma, Ltd",Released
9,Jan 3 2023 2:35PM,254198,19,ADV80010000,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,254197,Executing,1
48,254198,Released,26
49,254199,Released,1
50,254200,Released,6
51,254201,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254197,NaN,1.0,NaN
254198,NaN,NaN,26.0
254199,NaN,NaN,1.0
254200,NaN,NaN,6.0
254201,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,55.0,4.0,3.0
254104,0.0,0.0,1.0
254108,0.0,1.0,23.0
254111,0.0,53.0,0.0
254117,0.0,37.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254090,55,4,3
254104,0,0,1
254108,0,1,23
254111,0,53,0
254117,0,37,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,55,4,3
1,254104,0,0,1
2,254108,0,1,23
3,254111,0,53,0
4,254117,0,37,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254090,55,4,3
1,254104,,,1
2,254108,,1,23
3,254111,,53,
4,254117,,37,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 3 2023 2:42PM,254201,10,Ivaoes Animal Health
1,Jan 3 2023 2:41PM,254200,10,"Nextsource Biotechnology, LLC"
7,Jan 3 2023 2:36PM,254199,19,"AdvaGen Pharma, Ltd"
8,Jan 3 2023 2:35PM,254198,19,"AdvaGen Pharma, Ltd"
34,Jan 3 2023 2:07PM,254197,22,"NBTY Global, Inc."
35,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC"
46,Jan 3 2023 1:40PM,254193,10,"Senseonics, Incorporated"
47,Jan 3 2023 1:36PM,254192,10,Emerginnova
48,Jan 3 2023 1:32PM,254191,20,"Exact-Rx, Inc."
49,Jan 3 2023 1:26PM,254189,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 3 2023 2:42PM,254201,10,Ivaoes Animal Health,,,1
1,Jan 3 2023 2:41PM,254200,10,"Nextsource Biotechnology, LLC",,,6
2,Jan 3 2023 2:36PM,254199,19,"AdvaGen Pharma, Ltd",,,1
3,Jan 3 2023 2:35PM,254198,19,"AdvaGen Pharma, Ltd",,,26
4,Jan 3 2023 2:07PM,254197,22,"NBTY Global, Inc.",,1,
5,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC",,,11
6,Jan 3 2023 1:40PM,254193,10,"Senseonics, Incorporated",,,1
7,Jan 3 2023 1:36PM,254192,10,Emerginnova,,,1
8,Jan 3 2023 1:32PM,254191,20,"Exact-Rx, Inc.",,1,
9,Jan 3 2023 1:26PM,254189,10,ISDIN Corporation,,,29


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 2:42PM,254201,10,Ivaoes Animal Health,1,,
1,Jan 3 2023 2:41PM,254200,10,"Nextsource Biotechnology, LLC",6,,
2,Jan 3 2023 2:36PM,254199,19,"AdvaGen Pharma, Ltd",1,,
3,Jan 3 2023 2:35PM,254198,19,"AdvaGen Pharma, Ltd",26,,
4,Jan 3 2023 2:07PM,254197,22,"NBTY Global, Inc.",,1,
5,Jan 3 2023 1:59PM,254195,15,"Virtus Pharmaceuticals, LLC",11,,
6,Jan 3 2023 1:40PM,254193,10,"Senseonics, Incorporated",1,,
7,Jan 3 2023 1:36PM,254192,10,Emerginnova,1,,
8,Jan 3 2023 1:32PM,254191,20,"Exact-Rx, Inc.",,1,
9,Jan 3 2023 1:26PM,254189,10,ISDIN Corporation,29,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 2:42PM,254201,10,Ivaoes Animal Health,1,,
1,Jan 3 2023 2:41PM,254200,10,"Nextsource Biotechnology, LLC",6,,
2,Jan 3 2023 2:36PM,254199,19,"AdvaGen Pharma, Ltd",1,,
3,Jan 3 2023 2:35PM,254198,19,"AdvaGen Pharma, Ltd",26,,
4,Jan 3 2023 2:07PM,254197,22,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 2:42PM,254201,10,Ivaoes Animal Health,1.0,NaN,NaN
1,Jan 3 2023 2:41PM,254200,10,"Nextsource Biotechnology, LLC",6.0,NaN,NaN
2,Jan 3 2023 2:36PM,254199,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
3,Jan 3 2023 2:35PM,254198,19,"AdvaGen Pharma, Ltd",26.0,NaN,NaN
4,Jan 3 2023 2:07PM,254197,22,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 3 2023 2:42PM,254201,10,Ivaoes Animal Health,1.0,0.0,0.0
1,Jan 3 2023 2:41PM,254200,10,"Nextsource Biotechnology, LLC",6.0,0.0,0.0
2,Jan 3 2023 2:36PM,254199,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
3,Jan 3 2023 2:35PM,254198,19,"AdvaGen Pharma, Ltd",26.0,0.0,0.0
4,Jan 3 2023 2:07PM,254197,22,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6862191,534.0,120.0,2.0
12,762427,0.0,22.0,0.0
15,1016697,126.0,0.0,0.0
19,1779172,35.0,10.0,55.0
20,762444,2.0,1.0,0.0
21,254144,1.0,0.0,0.0
22,254197,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6862191,534.0,120.0,2.0
1,12,762427,0.0,22.0,0.0
2,15,1016697,126.0,0.0,0.0
3,19,1779172,35.0,10.0,55.0
4,20,762444,2.0,1.0,0.0
5,21,254144,1.0,0.0,0.0
6,22,254197,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,534.0,120.0,2.0
1,12,0.0,22.0,0.0
2,15,126.0,0.0,0.0
3,19,35.0,10.0,55.0
4,20,2.0,1.0,0.0
5,21,1.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,534.0
1,12,Released,0.0
2,15,Released,126.0
3,19,Released,35.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,2.0,0.0,0.0,55.0,0.0,0.0,0.0
Executing,120.0,22.0,0.0,10.0,1.0,0.0,1.0
Released,534.0,0.0,126.0,35.0,2.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,2.0,0.0,0.0,55.0,0.0,0.0,0.0
1,Executing,120.0,22.0,0.0,10.0,1.0,0.0,1.0
2,Released,534.0,0.0,126.0,35.0,2.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,2.0,0.0,0.0,55.0,0.0,0.0,0.0
1,Executing,120.0,22.0,0.0,10.0,1.0,0.0,1.0
2,Released,534.0,0.0,126.0,35.0,2.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()